# Pointer Networks: Arquitectura Autoregresiva para Selección de Índices

**Definición**: Un **Pointer Network** es un modelo de red neuronal basado en atención que genera como salida posiciones (índices) de la entrada, en lugar de valores continuos o categorías fijas. Su decoder es **autoregresivo**, es decir, cada paso depende de la salida previa y del estado interno, permitiendo secuencialmente “apuntar” a elementos de la secuencia de entrada.

**Referencia**: Oriol Vinyals, Meire Fortunato & Navdeep Jaitly (2015). ArXiv: [1506.03134](https://arxiv.org/abs/1506.03134)


## Problema: Suma de subconjunto mediante selección secuencial

**Objetivo**: Dados:
- Secuencia $X = [x_1, x_2, \dots, x_n]$
- Valor objetivo $S$

Encontrar un subconjunto de índices $\{i_1, \dots, i_k\}$ tal que:
```math
\sum_{j=1}^k x_{i_j} \approx S
```

**Desafío**: El espacio de búsqueda es exponencial $O(2^n)$.

**Estrategia**: Usar un Pointer Network autoregresivo como aproximador rápido.


## 1. Generación de Datos (`data.py`)

In [1]:
import numpy as np

def generate_sequence_target_data(num_samples=1000, seq_len=10, max_val=20):
    """
    Crea un dataset de ejemplo:
    - `num_samples`: cuántas secuencias generar.
    - `seq_len`: longitud de cada secuencia.
    - `max_val`: valor máximo aleatorio en la secuencia.
    Retorna: inputs, outputs (índices), target_sums.
    """
    inputs, outputs, target_sums = [], [], []
    for _ in range(num_samples):
        # Secuencia aleatoria de enteros 1..max_val
        seq = np.random.randint(1, max_val, size=seq_len)
        # Elegimos aleatoriamente cuántos índices tomamos (1 a seq_len-1)
        num_choices = np.random.randint(1, seq_len)
        subset_indices = sorted(
            np.random.choice(seq_len, size=num_choices, replace=False).tolist()
        )
        # Suma objetivo basada en los índices seleccionados
        target_sum = int(seq[subset_indices].sum())

        inputs.append(seq)
        outputs.append(subset_indices)
        target_sums.append(target_sum)
    return inputs, outputs, target_sums

# Ejemplo de uso
if __name__ == "__main__":
    inputs, outputs, sums = generate_sequence_target_data(5, seq_len=10)
    print("Ejemplo Inputs:", inputs)
    print("Ejemplo Outputs:", outputs)
    print("Ejemplo Sums:", sums)

Ejemplo Inputs: [array([ 4, 13,  6,  7,  2, 18, 15, 17,  4,  9]), array([14, 18, 12,  1, 14, 13,  1, 11,  4, 10]), array([ 6, 14, 12,  7, 19,  2,  6,  2, 15, 12]), array([11, 19,  2,  3,  2,  4,  7, 11, 16,  9]), array([11, 12,  5, 14, 17, 19,  9, 11, 13,  6])]
Ejemplo Outputs: [[0, 3, 4, 5, 6, 7], [1, 2, 3, 6, 7, 8, 9], [1, 3, 7, 8], [0, 1, 2, 3, 6, 7, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8]]
Ejemplo Sums: [63, 57, 38, 62, 111]


**Explicación**: Generamos suficientes muestras para entrenamiento; 

`seq_len=10` equilibra complejidad y velocidad; 

`max_val=20` da variabilidad moderada.

## 2. Definición del Modelo (`model.py`)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AutoregressivePointerNet(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=128):
        super().__init__()
        # LSTM encoder para procesar la secuencia de entrada
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        # LSTMCell para el decoder autoregresivo, paso a paso
        self.decoder_cell = nn.LSTMCell(input_dim, hidden_dim)
        # Proyecciones lineales para calcular atención
        self.W1 = nn.Linear(hidden_dim, hidden_dim)
        self.W2 = nn.Linear(hidden_dim, hidden_dim)
        self.vt = nn.Linear(hidden_dim, 1)

    def forward(self, x, target_len):
        # x: (batch, seq_len, input_dim)
        batch_size, seq_len, _ = x.size()
        enc_out, (h, c) = self.encoder(x)  # enc_out: (batch, seq, hidden_dim)
        mask = torch.zeros(batch_size, seq_len, device=x.device)
        pointers, log_probs = [], []

        # Entrada inicial: vector cero (no apuntado previo)
        dec_input = torch.zeros(batch_size, x.size(-1), device=x.device)
        hx, cx = h[0], c[0]  # hidden y cell state inicial del decoder

        for _ in range(target_len):
            # Un paso de decoder
            hx, cx = self.decoder_cell(dec_input, (hx, cx))

            # Atención: calculamos scores entre hx y cada enc_out[i]
            query = self.W2(hx).unsqueeze(1)      # (batch, 1, hidden)
            keys = self.W1(enc_out)               # (batch, seq, hidden)
            scores = self.vt(torch.tanh(keys + query)).squeeze(-1)  # (batch, seq)

            # Evitamos repetir índices seleccionados
            scores = scores.masked_fill(mask.bool(), float('-inf'))
            log_prob = F.log_softmax(scores, dim=1)

            idx = log_prob.argmax(dim=1)         # elegimos el puntero
            pointers.append(idx)
            log_probs.append(log_prob[torch.arange(batch_size), idx])

            # Actualizamos máscara y próxima entrada para decoder
            mask.scatter_(1, idx.unsqueeze(1), 1)
            dec_input = x[torch.arange(batch_size), idx]

        return torch.stack(pointers, dim=1), torch.stack(log_probs, dim=1)

**Explicación**: 

`hidden_dim=128` ofrece capacidad suficiente de representación sin ser demasiado lento; 

usamos `LSTMCell` para control manual en cada paso.


## 3. Evaluación (`evaluate.py`)

In [3]:
import time

def evaluate_model(model, inputs, outputs, target_sums):
    """
    Calcula MAE y tiempo medio de inferencia sobre datos de prueba.
    """
    model.eval()
    total_error, total_time = 0.0, 0.0
    n = len(inputs)

    with torch.no_grad():
        for seq, subset, tgt in zip(inputs, outputs, target_sums):
            x = torch.FloatTensor(seq).unsqueeze(0).unsqueeze(-1)
            start = time.time()
            pred_idx, _ = model(x, target_len=len(subset))
            elapsed = time.time() - start

            # Calculamos suma predicha y error absoluto
            pred = pred_idx[0].tolist()
            pred_sum = sum(seq[j] for j in pred)
            total_error += abs(pred_sum - tgt)
            total_time += elapsed

    mae = total_error / n
    avg_time = total_time / n
    return mae, avg_time

**Explicación**: 

Usamos MAE porque medimos distancia en valor continuo; 

Además medimos tiempo para evaluar la eficiencia real.


## 4. Entrenamiento (`train.py`)

In [4]:
import torch


def train_model(model, optimizer, data, val_data=None, num_epochs=10):
    train_in, train_out, train_sum = data
    for epoch in range(num_epochs):
        model.train()
        loss_total = 0.0
        for seq, subset in zip(train_in, train_out):
            x = torch.FloatTensor(seq).unsqueeze(0).unsqueeze(-1)
            y = torch.LongTensor(subset)
            optimizer.zero_grad()
            _, log_probs = model(x, target_len=len(y))
            loss = -log_probs.sum()  # cross-entropy positional
            loss.backward()
            optimizer.step()
            loss_total += loss.item()

        avg_loss = loss_total / len(train_in)
        if val_data:
            mae, avg_time = evaluate_model(model, *val_data)
            print(f"Epoch {epoch+1}/{num_epochs} — Loss: {avg_loss:.4f} — MAE: {mae:.4f} — Time: {avg_time*1000:.2f} ms")
        else:
            print(f"Epoch {epoch+1}/{num_epochs} — Loss: {avg_loss:.4f}")

**Explicación**: 

`num_epochs=10` es un compromiso entre calidad y tiempo; podemos ajustar según la convergencia.


## 5. Prueba y Resultados (`demo.py`)

In [ ]:
# Generación de datos
seq_len = 10
train_data = generate_sequence_target_data(200, seq_len)
val_data   = generate_sequence_target_data(50, seq_len)
val_tuple  = (val_data[0], val_data[1], val_data[2])

# Inicializar modelo y optimizador
model = AutoregressivePointerNet(input_dim=1, hidden_dim=64)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Entrenamiento con validación	rain_model(model, optimizer, train_data, val_data=val_tuple, num_epochs=15)

# Evaluación final
test_data = generate_sequence_target_data(5, seq_len)
mae, t_inf = evaluate_model(model, test_data[0], test_data[1], test_data[2])
print(f"\nTest MAE: {mae:.4f}, Inference time: {t_inf*1000:.2f} ms")

# Ejemplos de inferencia final
print("\nEjemplos finales:")
for seq, subset, tgt in zip(*test_data):
    x = torch.FloatTensor(seq).unsqueeze(0).unsqueeze(-1)
    pred_idx, _ = model(x, target_len=len(subset))
    pred = pred_idx[0].tolist()
    print(f"Seq: {seq}")
    print(f"  GT: {subset} (sum={tgt})")
    print(f"  Pred: {pred} (sum={sum(seq[j] for j in pred)})")
    print("-"*30) 


Test MAE: 12.4000, Inference time: 23.50 ms

Ejemplos finales:
Seq: [ 4 14 17  1  5  4 14  8 16 11]
  GT: [0, 1, 2, 3, 4, 5, 6, 8, 9] (sum=86)
  Pred: [0, 3, 5, 4, 1, 7, 2, 6, 9] (sum=78)
------------------------------
Seq: [13 10  3 17 13  5  3  5  1  5]
  GT: [1, 3, 4, 6, 8] (sum=44)
  Pred: [8, 0, 2, 9, 6] (sum=25)
------------------------------
Seq: [16  7 15 12 14  6 12 12  6  8]
  GT: [0, 1, 3, 5, 8, 9] (sum=55)
  Pred: [0, 1, 5, 8, 2, 9] (sum=58)
------------------------------
Seq: [17 11 19  6  3 19  8 14  9 17]
  GT: [0, 1, 2, 4, 5, 6, 7, 8, 9] (sum=117)
  Pred: [0, 4, 3, 1, 6, 2, 8, 5, 7] (sum=106)
------------------------------
Seq: [ 4  7 14  4 18 17 17 10 15 16]
  GT: [0, 6, 8] (sum=36)
  Pred: [0, 1, 3] (sum=15)
------------------------------
